In [1]:
%matplotlib inline
import sys
sys.path.append("/home/mette/github/implicitmodules")


# Sheared bunny

3D meshes registration using implicit modules of order 1. Normal frames and growth factor are optimized.



Import relevant Python modules.




In [2]:
import sys
sys.path.append("/home/mette/github/pytorch")

import torch
import meshio


import imodal

torch.set_default_dtype(torch.float64)
imodal.Utilities.set_compute_backend('keops')
device = 'cuda:0'
#device = 'cpu'

Load source and target data.




In [3]:
data_folder = "/home/mette/github/implicitmodules/data/"
source_mesh = meshio.read(data_folder+"bunny.ply")
target_mesh = meshio.read(data_folder+"bunny_shear.ply")

source_points = torch.tensor(source_mesh.points, dtype=torch.get_default_dtype())
target_points = torch.tensor(target_mesh.points, dtype=torch.get_default_dtype())
source_triangles = torch.tensor(source_mesh.cells_dict['triangle'], dtype=torch.long)
target_triangles = torch.tensor(target_mesh.cells_dict['triangle'], dtype=torch.long)

Rescaling source and target.




In [4]:
scale_factor = 100.
source_points = scale_factor*(source_points - torch.mean(source_points, dim=0))
target_points = scale_factor*(target_points - torch.mean(target_points, dim=0))

Generation of implicit module of order 1: points positions, initial growth
factor and normal frames.




In [5]:
# Defining an AABB around the source
aabb_source = imodal.Utilities.AABB.build_from_points(1.8*source_points)

# Generation of growth points
implicit1_density = 0.1
implicit1_points = imodal.Utilities.fill_area_uniform_density(imodal.Utilities.area_convex_hull, aabb_source, implicit1_density, scatter=1.8*source_points)

# Placeholders for growth factor and normal frames
implicit1_r = torch.empty(implicit1_points.shape[0], 3, 3)
implicit1_c = torch.empty(implicit1_points.shape[0], 3, 1)

# Initial growth factor constants
growth_constants = torch.tensor([[[1.], [1.], [1.]]], requires_grad=True, device=device)

# Initial normal frames angles. Normal frames are rotation matrices and thus defined by 3 angles.
angles = torch.zeros(implicit1_points.shape[0], 3, requires_grad=True, device=device)

Create the deformation model with a combination of 3 modules : implicit module
of order 1 (growth model), implicit module of order 0 (small corrections), global translation
and a large scale rotation.




Create and initialize the global translation module.




In [6]:
global_translation = imodal.DeformationModules.GlobalTranslation(3, coeff=10.)

Create and initialize the growth module.




In [7]:
sigma1 = 2.5/implicit1_density**(1/3)

implicit1 = imodal.DeformationModules.ImplicitModule1(3, implicit1_points.shape[0], sigma1, implicit1_c, nu=1000., gd=(implicit1_points, implicit1_r), coeff=0.001)

print("{} points for the implicit module of order 1.".format(implicit1_points.shape[0]))

217 points for the implicit module of order 1.


Create and initialize the local translations module.




In [8]:
implicit0_density = 0.25
sigma0 = 2./implicit0_density**(1/3)

print(sigma0)

implicit0_points = imodal.Utilities.fill_area_uniform_density(imodal.Utilities.area_convex_hull, aabb_source, implicit0_density, scatter=1.8*source_points)

implicit0 = imodal.DeformationModules.ImplicitModule0(3, implicit0_points.shape[0], sigma0, nu=1., gd=implicit0_points, coeff=1000.)

print("{} points for the implicit module of order 0.".format(implicit0_points.shape[0]))

3.1748021039363987
884 points for the implicit module of order 0.


Create and initialize the local large scale rotation.




In [9]:
rotation = imodal.DeformationModules.LocalRotation(3, 30., gd=torch.tensor([[0., 0., 0.], [0., 0., 1.]], device=device, requires_grad=True), backend='torch', coeff=10.)

Define our growth factor model.




In [10]:
# Function that computes normal frames from angles.
def compute_basis(angles):
    rot_x = imodal.Utilities.rot3d_x_vec(angles[:, 0])
    rot_y = imodal.Utilities.rot3d_y_vec(angles[:, 1])
    rot_z = imodal.Utilities.rot3d_z_vec(angles[:, 2])
    return torch.einsum('nik, nkl, nlj->nij', rot_z, rot_y, rot_x)


# Function that computes growth factor from growth factor constants.
def compute_growth(growth_constants):
    return growth_constants.repeat(implicit1_points.shape[0], 1, 1)


# Callback used by the registration model to compute the new growth factor
# and normal frames.
def precompute(init_manifold, modules, parameters, _):
    init_manifold[1].gd = (init_manifold[1].gd[0], compute_basis(parameters['growth']['params'][0]))
    modules[1].C = compute_growth(parameters['growth']['params'][1])

Define deformables used by the registration model.




In [11]:
deformable_source = imodal.Models.DeformableMesh(source_points, source_triangles)
deformable_target = imodal.Models.DeformableMesh(target_points, target_triangles)

deformable_source.to_device(device)
deformable_target.to_device(device)

Define the registration model.




In [12]:
sigmas_varifold = [1., 5., 15.]
attachment = imodal.Attachment.VarifoldAttachment(3, sigmas_varifold)

model = imodal.Models.RegistrationModel(deformable_source, [implicit1, implicit0, global_translation, rotation], [attachment], fit_gd=None, lam=100., precompute_callback=precompute, other_parameters={'growth': {'params': [angles, growth_constants]}})
model.to_device(device)

Fitting using Torch LBFGS optimizer.




In [ ]:
shoot_solver = 'euler'
shoot_it = 10
costs = {}
fitter = imodal.Models.Fitter(model, optimizer='torch_lbfgs')

fitter.fit(deformable_target, 500, costs=costs, options={'shoot_solver': shoot_solver, 'shoot_it': shoot_it, 'line_search_fn': 'strong_wolfe', 'history_size': 500})

[pyKeOps] Initializing build folder for dtype=float64 and lang=torch in /home/mette/.cache/pykeops-1.5-cpython-38 ... 

/home/mette/anaconda3/envs/py38/lib/python3.8/site-packages/torchdiffeq/_impl/misc.py:296: UserWarning: t is not on the same device as y0. Coercing to y0.device.
  warnings.warn("t is not on the same device as y0. Coercing to y0.device.")


done.
[pyKeOps] Compiling libKeOpstorcha05b4f18c1 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(TensorProd(-S*Exp(-S*SqNorm2(x - y)/IntCst(2))*(x - y), p),0)
       aliases: x = Vi(0,3); y = Vj(1,3); p = Vj(2,3); S = Pm(3,1); 
       dtype  : float64
... 
[pyKeOps] Compiling pybind11 template libKeOps_template_607a9d303c in /home/mette/.cache/pykeops-1.5-cpython-38 ... done.
Done.
[pyKeOps] Compiling libKeOpstorch8490a82e5d in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(TensorDot(-S*Exp(-S*SqNorm2(x - y)/IntCst(2))*(S*TensorDot(x-y, x-y, Ind(3), Ind(3), Ind(), Ind())-eye), p, Ind(3, 3), Ind(3, 3), Ind(1),Ind(1)),0)
       aliases: x = Vi(0,3); y = Vj(1,3); p = Vj(2,9); eye = Pm(3,9); S = Pm(4,1); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorch8ec0491cf5 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(TensorDot(TensorDot((-S*Exp(-S*SqNorm2(x_i - y_j)*IntInv(2))*(S*TensorDot(x_i -

/home/mette/github/implicitmodules/imodal/DeformationModules/ImplicitModule1.py:239: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:766.)
  c, _ = torch.solve(torch.mm(aq.t(), tlambdas.view(-1, 1)), aqkiaq)


Done.
[pyKeOps] Compiling libKeOpstorchbc5b3658a1 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(TensorDot(S*Exp(-S*SqNorm2(x - y)/IntCst(2))*(x - y), p, Ind(3), Ind(3, 3), Ind(0), Ind(1)),0)
       aliases: x = Vi(0,3); y = Vj(1,3); p = Vj(2,9); S = Pm(3,1); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorch348c2b0896 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(Exp(-S*SqNorm2(x - y)/IntCst(2))*X,0)
       aliases: x = Vi(0,3); y = Vj(1,3); X = Vj(2,3); S = Pm(3,1); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorchdd5cba9f93 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction((Exp(-S*SqNorm2(x - y)/IntCst(2))*px | py),1)
       aliases: x = Vi(0,3); y = Vj(1,3); px = Vi(2,3); py = Vj(3,3); S = Pm(4,1); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorcheb05f2a734 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Grad_WithSavedForward(

Done.
[pyKeOps] Compiling libKeOpstorch8175167ba5 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Grad_WithSavedForward(Sum_Reduction(TensorDot(TensorDot((-S*Exp(-S*SqNorm2(x_i - y_j)*IntInv(2))*(S*TensorDot(x_i - y_j, x_i - y_j, Ind(3), Ind(3), Ind(), Ind()) - eye)), A, Ind(3, 3), Ind(3, 3, 6, 6), Ind(0, 1), Ind(0, 1)), X, Ind(6, 6), Ind(6), Ind(0), Ind(0)),0), Var(1,3,1), Var(6,6,0), Var(7,6,0))
       aliases: x_i = Vi(0,3); y_j = Vj(1,3); X = Vj(2,6); eye = Pm(3,9); S = Pm(4,1); A = Pm(5,324); Var(6,6,0); Var(7,6,0); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorch7c89b7e4a3 in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Sum_Reduction(Exp(-S*SqNorm2(x - y)/IntCst(2)) * Square((u|v))*p,0)
       aliases: x = Vi(0,3); y = Vj(1,3); u = Vi(2,3); v = Vj(3,3); p = Vj(4,1); S = Pm(5,1); 
       dtype  : float64
... 
Done.
[pyKeOps] Compiling libKeOpstorch5299a2ce4c in /home/mette/.cache/pykeops-1.5-cpython-38:
       formula: Grad_WithSave

Compute optimized deformation trajectory.




In [ ]:
import time
intermediates = {}
start = time.perf_counter()
with torch.autograd.no_grad():
    deformed = model.compute_deformed(shoot_solver, shoot_it, intermediates=intermediates)[0][0].detach()
print("Elapsed={elapsed}".format(elapsed=time.perf_counter()-start))

basis = compute_basis(angles.detach()).cpu()
C = compute_growth(growth_constants.detach()).cpu()
print(growth_constants.detach().cpu())

imodal.Utilities.export_mesh("results_implicit_bunny/source.ply", source_points.cpu(), source_triangles)
imodal.Utilities.export_mesh("results_implicit_bunny/target.ply", target_points.cpu(), target_triangles)
imodal.Utilities.export_implicit1_growth("results_implicit_bunny/growth.vtk", implicit1_points, C)
imodal.Utilities.export_point_basis("results_implicit_bunny/basis.vtk", implicit1_points, basis)
imodal.Utilities.export_mesh_points("results_implicit_bunny/implicit0_points.vtk", implicit0_points)

for i, inter in enumerate(intermediates['states']):
    imodal.Utilities.export_mesh("results_implicit_bunny/{}.ply".format(i), inter[0].gd.cpu(), source_triangles)

with open("results_implicit_bunny/model.txt", 'w') as f:
    f.write(str(model))

with open("results_implicit_bunny/intermediates.pt", 'wb') as f:
    torch.save(intermediates, f)